# Gestructureerde data: Complexere architecturen

In de voorgaande onderdelen heb je reeds gemerkt dat het verwerken van de input in het geval van gestructureerde data kan leiden tot een complexere netwerkarchitectuur.
In deze notebook gaan we dit in meer detail bestuderen en ook kijken naar de mogelijkheden aan de output-kant van het netwerk.

## Multi-modal modellen (meerdere inputs combineren voor 1 output)

Een multi-modal neuraal netwerk is een netwerk dat leert van meerdere soorten data (modaliteiten) tegelijk. Modaliteiten kunnen bijvoorbeeld zijn:

* Gestructureerde data (sensorwaarden, tabellen, tijdreeksen)
* Beelden (RGB-foto's, video frames)
* Tekst (reviews, captions, transcripties)
* Audio (spraak, muziek)

Het doel is om de informatie uit verschillende bronnen te combineren zodat het model betere prestaties kan leveren dan wanneer je slechts één modaliteit gebruikt.

Elke modaliteit heeft vaak een gespecialiseerde encoder:
| Modaliteit    | Veelgebruikte encoders                               |
| ------------- | ---------------------------------------------------- |
| Gestructureerde | Fully connected layers, tabular transformers         |
| Beeld         | CNN (ResNet, EfficientNet), ViT (Vision Transformer) |
| Tekst         | RNN, LSTM, GRU, Transformers (BERT, GPT)             |
| Audio         | 1D-CNN, spectrogram + 2D-CNN, Wav2Vec                |

Er zijn verschillende manieren om features van de verscheidene modaliteiten te combineren:

* Early fusion: De ruwe data wordt gecombineerd vóór het netwerk (vaak lastig bij verschillende soorten data).
* Intermediate fusion: Features van verschillende subnetwerken worden gecombineerd in een middenlaag (meest gebruikelijk).
* Late fusion: Elke modaliteit geeft een voorspelling en die worden gecombineerd (bijv. gemiddelde, gewogen gemiddelde, of meta-classifier). Dit lijkt op de klassieke ensemble technieken die we vorig jaar gezien hebben.

Met pytorch ziet dit eruit als volgt:

Met de functial API van Keras ziet dit er dan uit als volgt

## Multi-task/output/head model

Een multi-task neuraal netwerk is een netwerk dat meerdere gerelateerde taken tegelijk leert. In plaats van voor elke taak een afzonderlijk netwerk te trainen, wordt één netwerk gedeeld over taken. Dit kan leiden tot betere generalisatie, omdat het netwerk gedeelde representaties leert die nuttig zijn voor meerdere taken.

Voorbeelden van multi-task learning:

* Gezichtsherkenning: voorspellen van leeftijd, geslacht en emotie van hetzelfde gezicht.
* Tabulaire data: voorspellen van zowel een continu resultaat (regressie) als een categorie (classificatie) uit dezelfde features.
* Gezondheidszorg: voorspellen van meerdere symptomen of uitkomsten uit patiëntgegevens.

Een multi-task model heeft vaak gedeelde lagen (shared layers) die features leren die nuttig zijn voor alle taken, en taakspecifieke lagen (task-specific layers) die de uiteindelijke voorspelling voor elke taak maken. Dit resulteert inde volgende architectuur.

* De eerste lagen zijn gedeeld voor alle taken.
* Daarna heeft elke taak een eigen output layer.

Daarnaast moet er voldoende aandacht besteed worden aan de loss-functie van dit netwerk.
Omdat er nu meerdere taken zijn, zijn er ook meerdere fouten (1 per taak).
Om het netwerk correct te trainen moet je de loss-functies combineren tot een samengestelde lossfunctie (vermenigvuldig elke loss-functie met een factor die je kiest en tel ze op).
Het gewicht van de loss-functie is echter belangrijk voor goed learning en om een goede waarde te vinden kan het nodig zijn om met verschillende parameters te experimenteren.

Hieronder staat een eenvoudig feedforward multi-task model in PyTorch dat één regressie- en één classificatietaak tegelijk uitvoert.

Met Keras ziet dit eruit als volgt

## Visualisatie van model architecturen

Er zijn heel wat verschillende manieren om de netwerkarchitectuur te visualiseren om goed te begrijpen wat er gebeurd.
Hieronder staan er een aantal voorbeelden voor

**PYTORCH**

In [ ]:
# print model

In [ ]:
# print summary

In [ ]:
# print each layer

In [ ]:
# make figure of the architecture

In [ ]:
# with tensorboard

**KERAS**

## Oefening - multitask

Ga aan de slag met [de wine-quality dataset](https://www.kaggle.com/datasets/uciml/red-wine-quality-cortez-et-al-2009).
Je kan de dataset downloaden met kagglehub of manueel.

De op te lossen taken met deze dataset zijn:

* Taak 1: Voorspel de kwaliteit (quality) als integer score 0–10 → classificatie of regressie (kan als beiden)
* Taak 2: Voorspel of de wijn goed is of slecht (bijvoorbeeld quality >= 6) → binary classification

Voer hieronder de volgende stappen uit en train zowel een pytorch model als een keras model:

* Data inspecteren: bekijk statistieken, histogram van kwaliteit en good/bad labels.
* Preprocessing: normalisatie van features.
* Split train/test.
* Model bouwen:
  * Shared layers
  * Task-specific heads (regression + binary)
* Loss functies instellen (MSE voor regressie, BCE voor classificatie)
* Train model
* Evalueer prestaties op beide taken (MAE voor regressie, accuracy voor binary)

In [ ]:
# data voorbereiden

In [ ]:
# dataloaders

In [ ]:
# multi-task model

In [ ]:
# train the model

**Met Keras**

In [ ]:
# bouw en train model with keras